# Graph Coloring Problem

La presente implementación detalla en primera instancia la solución del problema de coloración de vertices en grafos mediante programación lineal entera. Sobre esta se generan distintas instancias del problema en base a los sets dispuestos por https://mat.tepper.cmu.edu/COLOR/instances.html, no se utilizaron la totalidad de los sets dispuestos debido a la alta cantidad de vertices que contenian parte de estos, lo cual provocaba una escalada en tiempos de ejecución muy elevada.

In [1]:
import Pkg
using JuMP
using GLPK
using Glob
using IJulia

## Función de evaluación graph_coloring
La siguiente función detalla el procedimiento de coloración mediante backtracking.

En esta se representan como variables de decición binarias sobre $g$ como, la cantidad de vertices $i\in(1,...,n)$ por los colores $c\in(1,...,\Delta(g))$, donde se tendrian las variables $x_{ij}$ con $n*\Delta(g)$, donde $\Delta(g)$ corresponde al grado maximo del grafo y es una cota superior que permite reducir el valor de $num_{colors}$ 

Las restricciones son las siguientes.
- Los vertices adyacentes al vertice v no pueden tener el mismo color c, tal que la suma de los $c$ utilizados por los $x_{v,c}$ adyacentes sea igual a 1 (unico color en el vecindario).
- La cantidad de colores utilizados no puede exceder la cantidad de vertices en el grafo.

Como función objetivo se tiene que minimizar la cantidad de vertices coloreados de distinta manera, representados mediante la suma las variables $x$

In [2]:
function graph_coloring(graph)
    num_nodes = length(graph)
    max_degree = 0
    for i in 1:num_nodes
        degree = sum(graph[i])
        if degree > max_degree
            max_degree = degree
        end
    end
    println("max_degree: ", max_degree)

    # Crear el modelo de programación lineal
    model = Model(GLPK.Optimizer)

    # Variables de decisión: x[i, j] indica si el nodo i tiene el color j
    @variable(model, x[1:num_nodes, 1:max_degree], Bin)

    # Restricción: cada nodo debe tener exactamente un color asignado
    for i in 1:num_nodes
        @constraint(model, sum(x[i, :]) == 1)
    end

    # Restricción: dos nodos adyacentes no pueden tener el mismo color
    for i in 1:num_nodes
        for j in 1:num_nodes
            if graph[i][j] == 1
                for k in 1:max_degree
                    @constraint(model, x[i, k] + x[j, k] <= 1)
                end
            end
        end
    end

    # Función objetivo: minimizar el número de colores utilizados
    @objective(model, Min, sum(x))

    # Resolver el modelo
    optimize!(model)

    # Obtener los resultados
    coloring = Dict()
    for i in 1:num_nodes
        for j in 1:max_degree
            if value(x[i, j]) == 1
                coloring[i] = j
                break
            end
        end
    end

    return maximum(values(coloring))
end

graph_coloring (generic function with 1 method)

## Clase GraphMinified

Utilizada para la lectura de ficheros, almacena el nombre del archivo, la cantidad de vertices y las aristas entre cada vertice como una lista de aristas.

In [3]:
struct GraphMinified
    name::AbstractString
    vertices::Int
    edges::Vector{Tuple{Int, Int}}
end

In [4]:
file_names = glob("./instancias/*.col")

graphs_files = []
for file_name in file_names
    open(file_name) do file
        lines = readlines(file)
        vertices = 0
        edges = []
        name = file_name[14:end]
        for line in lines
            if startswith(line, "p")
                vertices = parse(Int, split(line)[3])
            elseif startswith(line, "e")
                push!(edges, (parse(Int, split(line)[2]), parse(Int, split(line)[3])))
            end
        end
        graph = GraphMinified(name, vertices, edges)
        push!(graphs_files, graph)
    end
end

# ordenar por número de vértices
sort!(graphs_files, by = x -> x.vertices)

println("Número de grafos: ", length(graphs_files))
for graph in graphs_files
    println("Nombre: ", graph.name, " Número de vértices: ", graph.vertices, " Número de aristas: ", length(graph.edges))
end

Número de grafos: 57
Nombre: myciel3.col Número de vértices: 11 Número de aristas: 20
Nombre: myciel4.col Número de vértices: 23 Número de aristas: 71
Nombre: queen5_5.col Número de vértices: 25 Número de aristas: 320
Nombre: queen6_6.col Número de vértices: 36 Número de aristas: 580
Nombre: myciel5.col Número de vértices: 47 Número de aristas: 236
Nombre: queen7_7.col Número de vértices: 49

 Número de aristas: 952
Nombre: queen8_8.col Número de vértices: 64 Número de aristas: 1456
Nombre: huck.col Número de vértices: 74 Número de aristas: 602
Nombre: jean.col Número de vértices: 80 Número de aristas: 508
Nombre: queen9_9.col Número de vértices: 81 Número de aristas: 2112
Nombre: david.col Número de vértices: 87 Número de aristas: 812
Nombre: myciel6.col Número de vértices: 95 Número de aristas: 755
Nombre: queen8_12.col Número de vértices: 96 Número de aristas: 2736
Nombre: queen10_10.col Número de vértices: 100 Número de aristas: 2940
Nombre: games120.col Número de vértices: 120

 Número de aristas: 1276
Nombre: queen11_11.col Número de vértices: 121 Número de aristas: 3960
Nombre: miles1000.col Número de vértices: 128 Número de aristas: 6432
Nombre: miles1500.col Número de vértices: 128 Número de aristas: 10396
Nombre: miles250.col Número de vértices: 128 Número de aristas: 774
Nombre: miles500.col

 Número de vértices: 128 Número de aristas: 2340
Nombre: miles750.col Número de vértices: 128 Número de aristas: 4226
Nombre: anna.col Número de vértices: 138 Número de aristas: 986
Nombre: queen12_12.col Número de vértices: 144 Número de aristas: 5192
Nombre: queen13_13.col Número de vértices: 169 Número de aristas: 6656
Nombre: mulsol.i.3.col Número de vértices: 184 Número de aristas: 3916
Nombre: mulsol.i.4.col Número de vértices: 185 Número de aristas: 3946
Nombre: mulsol.i.5.col Número de vértices: 186 Número de aristas: 3973
Nombre: mulsol.i.2.col

 Número de vértices: 188 Número de aristas: 3885
Nombre: myciel7.col Número de vértices: 191 Número de aristas: 2360
Nombre: queen14_14.col Número de vértices: 196 Número de aristas: 8372
Nombre: mulsol.i.1.col Número de vértices: 197 Número de aristas: 3925
Nombre: zeroin.i.3.col Número de vértices: 206 Número de aristas: 3540
Nombre: zeroin.i.1.col Número de vértices: 211 Número de aristas: 4100
Nombre: zeroin.i.2.col Número de vértices: 211 Número de aristas: 3541
Nombre: queen15_15.col Número de vértices: 225 Número de aristas: 10360
Nombre: queen16_16.col Número de vértices: 256 Número de aristas: 12640
Nombre: school1_nsh.col Número de vértices: 352 Número de aristas: 14612
Nombre: school1.col Número de vértices: 385 Número de aristas: 19095
Nombre: fpsol2.i.3.col Número de vértices: 425 Número de aristas: 8688
Nombre: le450_15a.col Número de vértices: 450 Número de aristas: 8168
Nombre: le450_15b.col Número de vértices: 450 Número de aristas: 8169


Nombre: le450_15c.col Número de vértices: 450 Número de aristas: 16680
Nombre: le450_15d.col Número de vértices: 450 Número de aristas: 16750
Nombre: le450_25a.col Número de vértices: 450 Número de aristas: 8260
Nombre: le450_25b.col Número de vértices: 450 Número de aristas: 8263
Nombre: le450_25c.col Número de vértices: 450 Número de aristas: 17343
Nombre: le450_25d.col Número de vértices: 450 Número de aristas: 17425
Nombre: le450_5a.col Número de vértices: 450 Número de aristas: 5714
Nombre: le450_5b.col Número de vértices: 450 Número de aristas: 

5734
Nombre: le450_5c.col Número de vértices: 450 Número de aristas: 9803
Nombre: le450_5d.col Número de vértices: 450 Número de aristas: 9757
Nombre: fpsol2.i.2.col Número de vértices: 451 Número de aristas: 8691
Nombre: fpsol2.i.1.col Número de vértices: 496 Número de aristas: 11654
Nombre: homer.col Número de vértices: 561 Número de aristas: 3258
Nombre: inithx.i.3.col Número de vértices: 621 Número de aristas: 13969
Nombre: inithx.i.2.col Número de vértices: 645 Número de aristas: 13979
Nombre: inithx.i.1.col Número de vértices: 864 Número de aristas: 18707


In [5]:
# itera por cada instancia
results = []

for graph_file in graphs_files
    println("Trabajando Grafo: ", graph_file.name, " Número de vértices: ", graph_file.vertices, " Número de aristas: ", length(graph_file.edges))
    elapsed_time_data = Float64[]
    num_colors_data = Float64[]
    mem_size_data = Float64[]
    for i in 1:1
        graph = []
        for j in 1:graph_file.vertices
            push!(graph, zeros(Int, graph_file.vertices))
        end
        for edge in graph_file.edges
            graph[edge[1]][edge[2]] = 1
            graph[edge[2]][edge[1]] = 1
        end
        mem_size = Base.summarysize(graph)
        elapsed_time = @elapsed num_colors = graph_coloring(graph)
        push!(elapsed_time_data, elapsed_time)
        push!(num_colors_data, num_colors)
        push!(mem_size_data, mem_size)
    end
    # determina la media de los resultados
    elapsed_time = sum(elapsed_time_data) / length(elapsed_time_data)
    num_colors = sum(num_colors_data) / length(num_colors_data)
    mem_size = sum(mem_size_data) / length(mem_size_data)
    res = [graph_file.name, graph_file.vertices, length(graph_file.edges), elapsed_time, num_colors, mem_size]
    push!(results, res)
    println("Resultados: ", res)
end

Trabajando Grafo: myciel3.col Número de vértices: 11 Número de aristas: 20
max_degree: 

5


Resultados: 

Any[

"myciel3.col", 11, 20, 3.2583082, 4.0,

...

3_1

3.col", 169, 6656, 2268.4142042, 21.0, 236640.0]
Trabajando Grafo: mulsol.i.3.col Número de vértices: 184 Número de aristas: 3916


max_degree: 157


Resultados: Any["mulsol.i.3.col", 184,

 3916, 7261.0171194, 157.0, 279720.0]
Trabajando Grafo: mulsol.i.4.col Número de vértices: 185